Implementation of DBM

In [ ]:
import torch
import torch.nn as nn
class DBM(nn.Module):
    def __init__(self, vis_dim=784, hid_dims=[512, 256]):
        super().__init__()
        self.W1 = nn.Parameter(torch.randn(vis_dim, hid_dims[0]) * 0.1)
        self.W2 = nn.Parameter(torch.randn(hid_dims[0], hid_dims[1]) * 0.1)
    def sample(self, prob):
        return torch.bernoulli(prob)
    def forward(self, v):
        h1_p = torch.sigmoid(v @ self.W1)
        return h1_p

vis_dim, lr, epochs = 784, 0.01, 5
dbm = DBM(vis_dim)
opt = torch.optim.Adam(dbm.parameters(), lr=lr)
for epoch in range(epochs):
  data = torch.randint(0, 2, (32, vis_dim)).float()
  h1_p = dbm(data)
  if epoch == epochs - 1:
    print(f"Hidden Probabilities (h1_p) after training:\n", h1_p)

Hidden Probabilities (h1_p) after training:
 tensor([[0.0501, 0.1335, 0.6574,  ..., 0.7331, 0.1754, 0.8456],
        [0.4828, 0.1974, 0.2598,  ..., 0.1755, 0.6429, 0.5675],
        [0.6723, 0.0237, 0.4145,  ..., 0.0936, 0.6984, 0.2147],
        ...,
        [0.1698, 0.0627, 0.9257,  ..., 0.6022, 0.3282, 0.1354],
        [0.4189, 0.6550, 0.4878,  ..., 0.7339, 0.1326, 0.3714],
        [0.3066, 0.2138, 0.7921,  ..., 0.1113, 0.1922, 0.0228]],
       grad_fn=<SigmoidBackward0>)


Implementation of DBN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
class DBN(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes) - 1)])
    def forward(self, x):
        for layer in self.layers:
            x = torch.sigmoid(layer(x))
        return x
    def fine_tune(self, X, y, epochs=5, lr=0.01):
        self.fc = nn.Linear(self.layers[-1].out_features, len(torch.unique(torch.Tensor(y))))
        opt, loss_fn = optim.Adam(self.parameters(), lr=lr), nn.CrossEntropyLoss()
        data, targets = torch.Tensor(X), torch.LongTensor(y)
        for epoch in range(epochs):
            opt.zero_grad()
            loss = loss_fn(self.fc(self(data)), targets)
            loss.backward()
            opt.step()
        print("Weights after training (final layer):")
        print(self.fc.weight)
layer_sizes, X_train, y_train = [784, 512, 256, 128], torch.randn(100, 784), torch.randint(0, 10, (100,))
dbn = DBN(layer_sizes)
dbn.fine_tune(X_train, y_train, epochs=5, lr=0.01)

Weights after training (final layer):
Parameter containing:
tensor([[-0.0256,  0.0192,  0.0434,  ...,  0.0655,  0.0677,  0.0257],
        [-0.0652,  0.0549, -0.0662,  ...,  0.0254, -0.1072,  0.0074],
        [ 0.0005,  0.0143, -0.0665,  ...,  0.0539, -0.0244,  0.0661],
        ...,
        [ 0.0754,  0.0222,  0.0813,  ..., -0.0539, -0.0164, -0.0477],
        [-0.0282,  0.0503, -0.0869,  ..., -0.0123,  0.0241, -0.0376],
        [-0.0260, -0.0272,  0.0006,  ...,  0.0333, -0.0598, -0.0735]],
       requires_grad=True)
